# Homework 3 -- Cloropleth map of grants in Swiss universities

In [ ]:
# imports and all that
import pandas as pd
import requests
from bs4 import BeautifulSoup

### Data wrangling

In [ ]:
grants = pd.read_csv("data/P3_GrantExport.csv", sep=';', index_col=0)
grants.size
grants.shape

We'll try to reduce the dataset by removing the entries that are not in CH and irrelevant columns.
We also remove data where location cannot be inferred, i.e. which are missing both the institute and the university

These changes can be safely persisted in the file, as it will not influence the data we work with.

In [ ]:
grants = grants[grants.University.notnull()]
grants = grants.drop(axis=1, errors='ignore', labels=['Project Title', 'Project Title English',
            'Responsible Applicant', 'Funding Instrument', 'Funding Instrument Hierarchy', 'Keywords'])

# drop rows where location cannot be inferred
filter_no_location = ~((grants.University == "Nicht zuteilbar - NA") & (grants.Institution.isnull()))
grants = grants[filter_no_location]
grants.dtypes
grants.shape

In [ ]:
# Sum entries in the Approved Amount column are not numeric and such are being dropped
grants['Approved Amount'] = pd.to_numeric(grants['Approved Amount'], errors='coerce')
grants = grants[grants['Approved Amount'].isnull() == False]
grants.shape

In [ ]:
# Run if you want to update the csv
#grants.to_csv("data/P3_GrantExport.csv", sep=';') # keep the same sep

In [ ]:
import re
def getLocationTagFromUni(s):
    return re.findall(r'\S+', s)[-1]

In [ ]:
grants['Location Tag'] = grants['University'].apply(getLocationTagFromUni)
grants.head()

In [ ]:
# Grouped by 'University' and the total funds each gets 
grants_grouped_by_uni = grants.groupby(['Location Tag', 'University'])
grants_grouped_by_uni = grants_grouped_by_uni.agg({'Approved Amount':'sum'}).reset_index()
grants_grouped_by_uni.shape

Command line 'voodoo' magic to get the list of cantons from the json file 

In [ ]:
cantons =! cat "ch-cantons.topojson.json" | grep "id\":" | cut -d\" -f4

In [ ]:
len(cantons)

Use Google API to find the location corresponding to the university

In [ ]:
google_key="0"
url = "https://maps.googleapis.com/maps/api/place/textsearch/xml?key="+google_key+"&query="
def get_city(row):
    #if row.postalcode != None: 
    r = requests.get(url+row.University)
    print(r.text)
    place = BeautifulSoup(r.text, "xml")
    result = place.find('result')
    if result != None:
        city = result.find('formatted_address').text
        return city[city.index(',')+2:city.index(',')+6]
    
#grants_grouped_by_uni['postalcode'] = grants_grouped_by_uni.apply(get_city, axis=1)
#grants_grouped_by_uni.loc[grants_grouped_by_uni['Location Tag'] == "BE", 'postalcode'] = 3012
grants_grouped_by_uni.head()

In [ ]:
frame = grants_grouped_by_uni.iloc[0:5]
#frame['postalcode'] = frame.apply(get_city, axis=1)
frame

TODO:

* we got 77 location tags and 26 cantons
* so the non-canton locations need to be mapped to cantons
* identify the 'how'
* also split NPO